![SUSS](https://www.suss.edu.sg/Sitefinity/WebsiteTemplates/SUSS/App_Themes/SUSS/images/logo.png)

# ANL252 - Python for Data Analytics
### End-of-Course Assessment, July Semester 2021 
### Sharil Ali 

### Question 1 (a)

Design a program to carry out the following task

In [1]:
# part (i)
import numpy as np
import pandas as pd

# Read CSV file
ship = pd.read_csv("ship.csv")
# Check for missing values
ship.isnull().all() # Returns false for all tables

# Replace "." as missing values
ship = ship.replace(".", np.nan)

In [2]:
# part (ii)
# Where MS is month of service, T is Ship Type, A is year of construction, 
# P is period of operations
ship = ship.rename(columns = {"T": "types", "A": "c_years", "P": "o_periods", 
                              "MS": "s_months", "Y": "incidents"})

In [3]:
# part (iii) 

# Convert s_months and incidents columns to numeric type
ship[["s_months", "incidents"]] = ship[
    ["s_months", "incidents"]].apply(pd.to_numeric)

# Group by as per requirement, and display the averages for each group
shipgroup = ship.groupby(["types", "o_periods"]).agg({"s_months": "mean"
                                                    , "incidents": "mean"})
# Round the averages to the nearest integer
shipgroup = shipgroup[["s_months", "incidents"]].round()
shipgroup

s_months  incidents
types o_periods                     
1     1             911.0        3.0
      2            1689.0        8.0
2     1           26852.0       36.0
      2           14440.0       36.0
3     1             914.0        2.0
      2             862.0        1.0
4     1             296.0        1.0
      2             889.0        4.0
5     1             664.0        4.0
      2            1047.0        7.0

In [4]:
# part (iv) - Locate missing s_months and replace accoridng to requirement

# Convert shipgroup to dictionary to retrieve the average values
# As dictionary is a faster method to lookup for data repeatedly
ship_dict = shipgroup.to_dict()

# Find missing value for s_months and iterate using itertuples rather than 
# itterows as it is much more efficient, and replace value accordingly
for row in ship[ship["s_months"].isnull()].itertuples():
    type, o_period =  getattr(row, "types"), getattr(row, "o_periods")
    
    # Replace the value with reference to the row's type and o_period of dict
    # As we are accessing a single value at a time, .at is appropriate and
    # faster than .loc
    ship.at[row.Index, "s_months"] = ship_dict["s_months"][type, o_period]
    
# Similarly to the above, find and replace missing value for incidents
for row in ship[ship["incidents"].isnull()].itertuples():
    type, o_period =  getattr(row, "types"), getattr(row, "o_periods")
    
    # Replace the value with reference to the row's type and o_period of dict
    ship.at[row.Index, "incidents"] = ship_dict["incidents"][type, o_period]
ship

,types,c_years,o_periods,s_months,incidents
0,1,1,1,127.0,0.0
1,1,1,2,63.0,0.0
2,1,2,1,1095.0,3.0
3,1,2,2,1095.0,4.0
4,1,3,1,1512.0,6.0
5,1,3,2,3353.0,18.0
6,1,4,1,911.0,3.0
7,1,4,2,2244.0,11.0
8,2,1,1,44882.0,39.0
9,2,1,2,17176.0,29.0


In [5]:
# part (v) 
Y = pd.DataFrame(ship["incidents"])

### Question 1 (b)

In [6]:
# part (i) - convert the required columns into categorical variables
category_columns = ["types", "c_years", "o_periods"]
ship[category_columns] = ship[category_columns].astype("category")

In [7]:
# part (ii) - Convert all categorical variables to dummy variable
X = pd.get_dummies(ship[category_columns])

In [8]:
# part (iii) - perform log-transformation of these variables
log_s_months = round(np.log(ship.s_months),2)
ship["log_s_months"] = log_s_months
X["log_s_months"] = log_s_months

### Question 1 (c)
There are only 40 observations in the DataFrame, which is insufficient to split the dataset into training and testing datasets. 

If the DataFrame were to be split into training and testing datasets, both datasets would have insufficient data. 

Insufficient data in the training set would result in the ineffective ability of the model to learn the mapping of inputs and outputs.

The insufficient data in the testing dataset would result also in an ineffective evaluation of the models’ performance when testing. 

### Question 1 (d) - Export ship df as a new .csv and .db file

In [9]:
# Export as .csv
ship.to_csv("ship_prepared.csv", index=False)

# Export as .db 
import sqlite3
conn = sqlite3.connect("ship.db")
cur = conn.cursor()
ship.to_sql("ship_table", conn, if_exists="replace", index = False)

# Check .db file
#cur.execute("SELECT*FROM ship_table")
#cur.fetchall()

### Question 2 (a)

The corresponding module would be .linear_model.PoissonRegressor(). This module is for a generalized linear model that has a Poisson distribution. This module utilizes ‘log’ link function; where there is a natural fit for count variables that follow the Poisson distribution. 

This module has 6 parameters which are – alpha, fit_intercept, max_iter, tol, warm_start and verbose. 

The parameter alpha only accepts float input and the default value, if undeclared is set to 1. It is used to determine the regularization strength where the constant multiplies a penalty term. If the alpha is set to zero, it is equivalent to an unpenalized general linear model.

The parameter fit_intercept only accepts Boolean input – True or False, where the default value is set to True. This parameter affects the linear predictor where it determines whether the intercept or bias value is added to the regression coefficient for linear predictor. If it is set to True, it would add the intercept or bias value to the regression for linear predictor, else it would only take the coefficient value for the linear predictor.

The parameter max_iter take integer inputs only, where the default is set to 100. This parameter specifies the maximum number of iterations for the solver. 

The parameter warm_start takes in Boolean input only, where the default value is False. If there is a need to fit an estimator repeatedly on the same dataset, the parameter should be set to True as it would reuse aspects of model learnt from previous solution where the coefficient and intercept value is kept, initializing the new model in a subsequent call to fit. 

The verbose parameter only accepts integer value where the default is set to zero. This is to set verbosity for the Limited-memory Broyden–Fletcher–Goldfarb–Shanno (LBFGS) solver.

There are two function to set and retrieve the estimator – get_params() and set_params(). get_params() would return an array of all the parameters above that has been declared. set_params() would allow the user to declare the parameters of the estimator. 

The .fit() function fits the dataset to a generalized linear model where the parameters are X, y and sample_weight. X would be the training data, where it is array-like and a sparse matrix with the shape of (n_samples, n_features). Y would be the target values, where it is also array-like but with a shape of (n_samples). Sample_weight parameter accepts None or an array-like shape with positive values used as a representative of the observation. 

The .predict() function predicts the y values to the given X array values based on the existing generalized linear model. The parameter is X, known as samples, where it is array-like and a sparse matrix with the shape of (n_samples, n_features). 



### Question 2 (b)

In [10]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor()

# Get parameters of estimated model
clf.get_params()

# fit X and Y to get the coefficients
clf.fit(X, Y.incidents)

# Get the coefficients
coefficient = clf.coef_
# Create a DataFrame to present the coefficients with the corresponding labels
coefficient_df = pd.DataFrame(coefficient, X.columns)
coefficient_df.columns = ["Coefficient"]
coefficient_df

,Coefficient
types_1,0.049117
types_2,0.087891
types_3,-0.244508
types_4,-0.067887
types_5,0.175397
c_years_1,-0.245196
c_years_2,0.154951
c_years_3,0.176998
c_years_4,-0.086743
o_periods_1,-0.145163


### Question 2 (c) - Compute deviance without using score() of scikit-learn

The deviance should be calculated with the following equation:

$$ D=2\sum_{i=1}^{n}\left\{Y\log{\left[\frac{Y}{exp{\left(\mathbb{E}\left(Y\right)\right)}}\right]}-[Y-\exp{\left(\mathbb{E}\left(Y\right)\right)})\right\} $$

In [11]:

expected_df = pd.DataFrame(clf.predict(X), columns=["expected_incidents"])
# Merge Y and expected Y to calculate D through iteration
incidents_df = pd.concat([Y, expected_df], axis=1, join="outer")


a_Y = incidents_df["incidents"]
e_Y = incidents_df["expected_incidents"]
# Apply formula where incidents is not zero
incidents_df["calculations"] = a_Y*np.log(a_Y/e_Y) - (a_Y - e_Y).where(
    incidents_df["incidents"] != 0
)
# If incidents is zero, calculations equals to respective expected_incidents
incidents_df["calculations"] = incidents_df["calculations"].fillna(
    incidents_df["expected_incidents"])
D = 2*incidents_df["calculations"].sum()

print(f"The deviance of Y and its expected Y is {D:.2f}")

The deviance of Y and its expected Y is 52.33


/Users/sharilali/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
